### Notebook order

This notebook is the 1st notebook in milestone 1. 

In this notebook, we explore and decide what to do in the data cleaning process.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import missingno as msno

import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv("https://raw.githubusercontent.com/MohamedMostafa259/Customer-Churn-Prediction-and-Analysis/main/Data/train.csv")
train_copy = train.copy()

### Data Overview

In [ ]:
train_copy.sample(5, random_state=42)

**Comments:**

- What does `'xxxxxxxx'` mean in the `'referral_id'` column?

- The `'avg_frequency_login_days'` numeric column seems to contain values = `'Error'`

In [ ]:
train_copy.info()

In [ ]:
train_copy.isna().sum()[train_copy.isna().sum() > 0]

**Comments:**

- These columns contain NaNs:

	-	region_category               
	-	preferred_offer_types         
	-	points_in_wallet              

- The `'avg_frequency_login_days'` numeric column has type of `object`! (from previous comments cell, we found out that it seems to contain values = `'Error'`)

In [ ]:
train_copy.describe()

In [ ]:
train_copy['churn_risk_score'].value_counts(dropna=False).sort_index()

In [ ]:
train_copy.describe(include=object)

`gender` column has 3 unique categories!!

In [ ]:
train_copy['gender'].value_counts(dropna=False)

In [ ]:
train_copy.loc[train_copy['days_since_last_login'] < 0, 'days_since_last_login'] = np.nan
train_copy.loc[train_copy['avg_time_spent'] < 0, 'avg_time_spent'] = np.nan
train_copy.loc[train_copy['points_in_wallet'] < 0, 'points_in_wallet'] = np.nan

train_copy.loc[train_copy['churn_risk_score'] == -1, 'churn_risk_score'] = np.nan
train_copy.dropna(subset=['churn_risk_score'], inplace=True)

**Comments:**

- Negative values should be set to `np.nan` in these columns:

	-	days_since_last_login
	-	avg_time_spent
	-	points_in_wallet
	-	churn_risk_score (NaNs in the target: these rows needs to be dropped)

-	Target class has imbalanced class distribution

- `'gender'` column contains a category called `'Unknown'`

### Category value counts

In [ ]:
for col in train_copy.select_dtypes(include='object').columns:
	print(train_copy[col].value_counts(normalize=True, dropna=False))
	print('-'*50)

In [ ]:
train_copy.replace('?', np.nan, inplace=True)

In [ ]:
train_copy.loc[train_copy['avg_frequency_login_days'] == 'Error', 'avg_frequency_login_days'] = np.nan
train_copy['avg_frequency_login_days'] = train_copy['avg_frequency_login_days'].astype(float)
train_copy.loc[train_copy['avg_frequency_login_days'] < 0, 'avg_frequency_login_days'] = np.nan
train_copy['avg_frequency_login_days'].value_counts(dropna=False, normalize=True)

**Comments:**

-	Around $14\%$ of `'joined_through_referral'` and `'medium_of_operation'` columns = `'?'`
-	cols_to_drop ↓↓

	-	We need to drop unnecessary columns: `train.drop(columns=cols_to_drop, inplace=True)`

-	date_cols ↓↓

In [ ]:
cols_to_drop = ['customer_id', 'Name', 'security_no', 'referral_id']

# date_cols = [('date', 'date_format'), ...]
date_cols = [('joining_date', '%Y-%m-%d'), ('last_visit_time', '%H:%M:%S')]

### Check duplicates

In [ ]:
train_copy.duplicated(['customer_id']).sum()

#### Visualizing distributions

In [ ]:
train_copy.hist(bins=50, figsize=(10, 7))
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 7, figsize=(18, 4))
idx = 0
for col in train_copy.select_dtypes(include=np.number).columns:
	sns.kdeplot(train_copy[col], ax=axes[idx])
	idx += 1
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 7, figsize=(18, 4))
idx = 0
for col in train_copy.select_dtypes(include=np.number).columns:
	sns.boxplot(train_copy[col], ax=axes[idx])
	idx += 1
plt.tight_layout()
plt.show()

In [ ]:
# cols_with_outliers = ['avg_time_spent', 'avg_transaction_value', 'avg_frequency_login_days', 'points_in_wallet']

In [ ]:
fig, axes = plt.subplots(1, 2)
sns.violinplot(train_copy['avg_time_spent'], ax=axes[0])
# log1p(x) = log(x + 1): this avoids errors when x = 0
sns.violinplot(np.log1p(train_copy['avg_time_spent']), ax=axes[1])
plt.tight_layout()
plt.show()

**Comments:**

Some columns have non-normal and non-uniform distributions. Also, they have outliers; however, I will not cap these outliers because they may introduce an important pattern in the data, for example, if `avg_transaction_value` has some very high values due to VIP customers, capping them may remove valuable patterns.

So, I will apply log transformation to `avg_time_spent` to make it look more normal as it's right-skewed.

### Missing value analysis

Before imputing NaNs, we need to have a good understanding of how they are distributed in our dataset.

In [ ]:
missing_percent = train_copy.isna().mean() * 100  
missing_count = train_copy.isna().sum()           

plt.figure(figsize=(15, 6))
bars = plt.bar(missing_percent.index, missing_percent, color='skyblue')

# Annotate bars with both count and percentage
for bar, count, percent in zip(bars, missing_count, missing_percent):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'{int(count)}\n({percent:.1f}%)',
             ha='center', va='bottom', fontsize=8)

plt.xticks(rotation=90)
plt.ylabel('Missing Value Percentage')
plt.title('Missing Data: Count and Percentage per Column')
plt.tight_layout()
plt.show()


In [ ]:
train_copy['last_visit_time'] = pd.to_datetime(train_copy['last_visit_time'])
train_copy.sort_values('last_visit_time', inplace=True)
msno.matrix(train_copy)
plt.show()

In [ ]:
train_copy['joining_date'] = pd.to_datetime(train_copy['joining_date'])
train_copy.sort_values('joining_date', inplace=True)
msno.matrix(train_copy)
plt.show()

In [ ]:
msno.heatmap(train_copy)
plt.show()

**Comments:**

It's clear that there is no patterns in missingness (if some columns are missing together)

### Sanity check

`referral_id` should be populated only when `joined_through_referral` is "Yes"

In [ ]:
train_copy['referral_id'].isna().sum()

In [ ]:
wrong_rows = (train_copy['referral_id'] != 'xxxxxxxx') & (train_copy['joined_through_referral'] == 'No')
train_copy.loc[wrong_rows]

In [ ]:
wrong_rows = (train_copy['referral_id'] == 'xxxxxxxx') & (train_copy['joined_through_referral'] == 'Yes')
train_copy.loc[wrong_rows]

In [ ]:
train_copy.loc[wrong_rows].hist(bins=50, figsize=(10, 7))
plt.show()

In [ ]:
train_copy.loc[wrong_rows].describe(include=object)
# about 50% of them have 'complaint_status'='Not Applicable'

**Comments:**

There're lots of columns seems to have inconsistent data; how to deal with them?!

I prefer to not remove them because first we need to ask the data owners what it means when 'referral_id' is equal to 'xxxxxxxx'. 